In [477]:
import pandas as pd
import numpy as np
import scipy.io as sio
from scipy import stats

# Read in dorcs 


In [478]:
dorcs= pd.read_csv("foxa1_data/G2_dorcs_nopseudo.csv", sep = ",")

In [479]:
chroms = [i.split('_')[0] for i in dorcs['peak']]

In [480]:
dorcs['Chr'] = chroms

In [481]:
x = dorcs['Gene'].value_counts(dropna=True)
y = dorcs['Gene'].value_counts(dropna=True).index
dorclist = []
dorcnums = []
z = x.index
for i in range(len(x)):
    if x[i] >= 10:
        dorclist.append(z[i])
        dorcnums.append(x[i])
dorcs["region"] = dorcs['Chr'] + "_" + (dorcs["Start"].astype(int) + 1).astype(str) + "_" + dorcs["End"].astype(str)
#dorclist

# Read in counts of atac peaks: Normalize the peak matrix - log normalize using lognormalize functon in r before loading in ^^^

In [439]:
#reading in the cell by peak matrix
peaks = pd.read_csv("foxa1_data/bedpeaksfoxa1_new.bed", sep= "\t", header = None)
peaks.columns = ["chr", "start", "end"]
peaks["start"] = peaks["start"] + 1
peaks["region"] = peaks["start"].astype(str) + "_" + peaks["end"].astype(str)
peaks["total"] = peaks["chr"].astype(str) + "_" + peaks["region"].astype(str)


In [440]:
counts = sio.mmread("foxa1_data/normalizedpeaks.mtx")
cell_ids = pd.read_csv("foxa1_data/G2peakcelltypes.txt", header = 0, sep = "\t")
cell_idrna = pd.read_csv("foxa1_data/G2rnacelltypes.txt", header = 0, sep = "\t")


In [441]:
cell_ids = cell_ids['x']
cell_idrna = cell_idrna['x']
#cell_idrna = cell_ids['rna.bc']
#cell_ids = cell_ids['atac.bc']
#bedfile <- data(counts, package = "chromVAR")
counts.columns = cell_ids
counts.rows = peaks['total']

# find the dorc scores to create the dorc x cell matrix

In [442]:
#for each gene in dorclist
dorc_by_cell = []
for i in dorclist:
    #find all associated peak names from the table above,find them in the count matrix and sum their arrays into 1
    peak_names = dorcs[dorcs['Gene'] == i]['region'].tolist()
    geneind2 = [j for j, e in enumerate(counts.rows) if e in peak_names]
    corr = counts.tocsr()[geneind2, : ]
    corr = corr.toarray()
    corr = np.sum(corr, axis = 0)
    #print(corr)
    #add this to a new dorc_matrix with the same cell columns
    dorc_by_cell.append((i, corr))
    #print(i)

In [443]:
dorc_by_cell = pd.DataFrame(dorc_by_cell)
dorc_by_cell.head()

,0,1
0,Fam71a,"[0.8657228741326698, 1.3014683508071005, 2.146..."
1,Krt16,"[0.6212482011593523, 1.9624368524069649, 0.919..."
2,Mybbp1a,"[0.44559927179166714, 0.25800024558829626, 0.3..."
3,Nsmf,"[0.5178885753598412, 0.1696058526882664, 0.546..."
4,Smgc,"[0.17745273002214812, 0.1696058526882664, 0.15..."


In [444]:
dorc_by_cell.columns = ['gene', 'cell']
dorc_by_cell.index = dorc_by_cell['gene']
dorc_by_cell = dorc_by_cell[['cell']]

# read in rna counts

In [445]:
rna_expression = sio.mmread('foxa1_data/foxA1_rna_counts_G2.mtx')



In [446]:
to_meta = pd.read_csv('foxa1_data/G2_rna_features.tsv', sep = '\t', header = 0)

In [447]:
to_meta.head()

,x
1,Xkr4
2,Rp1
3,Mrpl15
4,Lypla1
5,Tcea1


In [448]:
to_meta.columns = ['gene']

In [449]:
rna_expression.rows = to_meta['gene']
rna_expression.columns = cell_idrna

# Create dictionary for indexes of atac cells and rna cells in the dorc cell matrix

In [450]:
todic = counts.columns
todic = dict(todic)
todic = {v: k for k, v in todic.items()}
todic2 = dict(rna_expression.columns)

todic2 = {v: k for k, v in todic2.items()}

#dictionary of cells and their atac index
atac_cells = todic
#dictionary of cells and their rna index
rna_cells = todic2

# read in the topic knn

In [451]:
topics = pd.read_csv('foxa1_data/cisTopic40Topicsoutput.tsv', delimiter = '\t')
topics.head()

,G2_multiome#TGAAACTGTTATCGAG-1,G2_multiome#GTTGCCCGTCGTAAAT-1,G2_multiome#CCGCAAGGTTAGCGTA-1,G2_multiome#CCATAATCAGTACCGT-1,G2_multiome#GCCAGGAAGCCGGTAC-1,G2_multiome#AACGGTAAGGCGCTAC-1,G2_multiome#CGCAATGTCACAGACT-1,G2_multiome#GAGCTAGCACAATTAC-1,G2_multiome#CGGTTGGGTTCCGGGA-1,G2_multiome#CATAGGTTCCACCCTG-1,...,G2_multiome#TATGACTCAGGATGGC-1,G2_multiome#CCAACATAGGAGTCTT-1,G2_multiome#GAAGTCAAGGAAACTG-1,G2_multiome#GATGCGACAAGGTACG-1,G2_multiome#AGTCTTGCAAGCGAGC-1,G2_multiome#ACCACATAGCTATTAG-1,G2_multiome#CGATTATCACGTGCTG-1,G2_multiome#CGGTTCCGTAAATTGC-1,G2_multiome#GCTAACAGTCGTTACT-1,G2_multiome#TACGCACCAAAGCGCA-1
Topic1,0.009151,0.042943,0.031055,0.034944,0.041689,0.036637,0.037729,0.058899,0.028316,0.028925,...,0.043758,0.028010,0.041684,0.004288,0.017191,0.022964,0.036079,0.001692,0.016341,0.001694
Topic2,0.030957,0.003242,0.009923,0.035081,0.031202,0.020575,0.002919,0.000761,0.020918,0.019800,...,0.045068,0.040371,0.085759,0.000580,0.008202,0.031405,0.001579,0.000896,0.003018,0.004299
Topic3,0.012864,0.033480,0.025569,0.022911,0.028848,0.025792,0.030448,0.044058,0.034982,0.024471,...,0.004038,0.004486,0.001767,0.050649,0.001011,0.014523,0.033649,0.008459,0.019255,0.026707
Topic4,0.023656,0.027409,0.015581,0.010588,0.014607,0.009411,0.032162,0.034472,0.014343,0.007039,...,0.000546,0.003289,0.000936,0.070584,0.001461,0.038357,0.020044,0.133061,0.027165,0.054325
Topic5,0.012855,0.036968,0.032262,0.029514,0.028255,0.024213,0.043390,0.030369,0.041684,0.033578,...,0.017132,0.029207,0.013410,0.056676,0.017191,0.064672,0.071064,0.018014,0.059638,0.104351


In [452]:
import sklearn.neighbors as sk
topicknn = sk.kneighbors_graph(topics.transpose(), mode='connectivity', n_neighbors=50)

In [453]:
topicknn.columns = cell_ids
topicknn.rows = cell_ids


In [454]:
topicknn

<5219x5219 sparse matrix of type '<class 'numpy.float64'>'
	with 260950 stored elements in Compressed Sparse Row format>

In [455]:
topicknn.rows = topicknn.rows.reset_index(drop = True)
topicknn.columns = topicknn.columns.reset_index(drop = True)

In [456]:
topicdict = {}
for i in range(len(topicknn.rows)):
    row = topicknn.tocsr()[i, :].toarray()
    indexes = []
    [indexes.append(j) for j in range(len(row[0])) if row[0][j] > 0]
    topicdict[topicknn.rows[i]] = indexes

# create rna x cell modified matrix

In [457]:
rna_expression

<18712x5219 sparse matrix of type '<class 'numpy.float64'>'
	with 21226087 stored elements in COOrdinate format>

In [458]:
rind = [j+1 for j, e in enumerate(rna_expression.rows.tolist()) if e in dorc_by_cell.index.tolist()]

In [459]:

rna_expression = rna_expression.tocsr()[rind, : ]
rna_expression.columns = cell_idrna

In [460]:

rna_expression.rows = to_meta['gene'][rind]
rna_by_cell = pd.DataFrame(rna_expression.todense())
rna_by_cell = rna_by_cell.values.tolist()

In [461]:

rna_by_cell2 = pd.DataFrame(rna_expression.rows)
rna_by_cell2 = rna_by_cell2.reset_index(drop=True)
rna_by_cell2['cell'] = [i for i in rna_by_cell]
rna_by_cell2.index = rna_by_cell2['gene']
rna_by_cell2 = rna_by_cell2[['cell']]
rna_by_cell2.head()

,cell
gene,
Fam178b,"[0.0, 1.9459101490553132, 2.0794415416798357, ..."
Map4k4,"[0.0, 0.0, 0.0, 0.6931471805599453, 0.0, 0.0, ..."
AI597479,"[0.0, 0.6931471805599453, 0.6931471805599453, ..."
Bzw1,"[0.0, 0.6931471805599453, 1.0986122886681096, ..."
Fcamr,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [193]:
#dorc_by_cell = pd.read_csv('dorc_by_cell.csv')

# smooth functions

In [462]:
def Avg(array, knn):
    toreturn = []
    for i in range(len(array)):
        val = topicknn.rows[i]
        inds = knn.get(val)
        total = int(array[i])
        for num in inds:
            total += array[num]
        total = (total/(1 + len(inds)) )
        toreturn.append(total)
    return toreturn

In [463]:
def smooth(dataframe, knn):
    count = 0
    #dataframe is a dorc x cell matrix or a rna x cell matrix
    #make sure if rna x cell matrix, we only use the dorc rnas 
    print(len(dataframe.index))
    newmat = []
    for key, value in dataframe.iterrows():
        rep = value[0]
        newrep = Avg(rep, knn)
        newmat.append((key, newrep))    
        
        #genename = key
        #geneind2 = [j for j, e in enumerate(tempy) if i == e]
        #corr = counts.tocsr()[geneind2, : ]
        #corr = corr.toarray()
        if count % 200 == 0:
            print(len(newmat))
        count += 1
    return(newmat)

# normalize our dorc matrix by peak counts - dont do this



In [ ]:
#curr = [0 for i in range(len(dorc_by_cell.iloc[0][0]))]
#for i in dorc_by_cell['cell']:
#    zipped_lists = zip(i, curr)
#    curr = [x + y for (x, y) in zipped_lists]

In [ ]:
#d2 = []
#for index, row in dorc_by_cell.iterrows():
#    zipped_lists = zip(row['cell'], curr)
#    tp = [np.log(((x / y)*10000)+1) for (x, y) in zipped_lists]
#    d2.append(tp)

#d2 = pd.Series(d2)
#d2.index = dorc_by_cell.index
#dorc_by_cell['cell'] = d2
#dorc_by_cell = dorc_by_cell[['cell']]
#dorc_by_cell.head()

# smooth the dorc scores

In [464]:
#smooth the dorc scores
smoothed_dorc = smooth(dorc_by_cell, topicdict)
#smooth the rna scores

260
1
201


# smooth the rna scores

In [465]:
smoothed_rna = smooth(rna_by_cell2, topicdict)

260
1
201


# get dorc scores and rna in usable formats

In [466]:
smoothed_rna = pd.DataFrame(smoothed_rna)
smoothed_rna.columns = ['gene', 'cell']
smoothed_dorc = pd.DataFrame(smoothed_dorc)
smoothed_dorc.columns = ['gene', 'cell']
revdorc = pd.DataFrame(cell_ids)

In [ ]:
#was trying with more identified dorcs >= 10

In [467]:
matsmoothvals = smoothed_dorc.cell.apply(pd.Series)

In [468]:
matsmoothvals.index = dorc_by_cell.index
matsmoothvals = matsmoothvals.reindex(dorc_by_cell.index)

In [476]:
matsmoothvals.head()

,0,1,2,3,4,5,6,7,8,9,...,5209,5210,5211,5212,5213,5214,5215,5216,5217,5218
gene,,,,,,,,,,,,,,,,,,,,,
Fam178b,1.491616,1.824243,1.865978,1.799586,1.796901,1.830486,1.720219,2.038765,1.660460,1.748329,...,1.856369,1.817774,1.957344,1.397603,2.022300,1.607572,1.850687,1.614656,1.486864,1.699164
Map4k4,0.054364,0.021541,0.081547,0.083856,0.048724,0.067956,0.027182,0.021541,0.067956,0.040773,...,0.027182,0.089497,0.130270,0.013591,0.000000,0.027182,0.021541,0.040773,0.013591,0.000000
AI597479,0.656463,0.643428,0.649865,0.811693,0.921158,0.708605,0.649573,0.515508,0.900329,0.835433,...,0.530076,0.512020,0.529186,0.815232,0.557236,0.896766,0.569827,0.638340,0.520060,0.452464
Bzw1,0.851849,0.859307,1.009986,1.107764,1.200532,0.966306,0.907786,0.757329,1.110300,1.129532,...,1.112185,0.840208,0.883291,0.760660,0.894612,1.131841,1.019013,0.875613,0.648975,0.941322
Fcamr,0.000000,0.000000,0.013591,0.000000,0.000000,0.000000,0.000000,0.000000,0.013591,0.000000,...,0.000000,0.021541,0.013591,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [470]:
matsmoothvals.to_csv('foxa1_data/smoothed_dorc_table_G2_40.csv')

In [471]:
matsmoothvals = smoothed_rna.cell.apply(pd.Series)

In [472]:
matsmoothvals.index = rna_by_cell2.index
matsmoothvals = matsmoothvals.reindex(rna_by_cell2.index)

In [475]:
matsmoothvals.head()

,0,1,2,3,4,5,6,7,8,9,...,5209,5210,5211,5212,5213,5214,5215,5216,5217,5218
gene,,,,,,,,,,,,,,,,,,,,,
Fam178b,1.491616,1.824243,1.865978,1.799586,1.796901,1.830486,1.720219,2.038765,1.660460,1.748329,...,1.856369,1.817774,1.957344,1.397603,2.022300,1.607572,1.850687,1.614656,1.486864,1.699164
Map4k4,0.054364,0.021541,0.081547,0.083856,0.048724,0.067956,0.027182,0.021541,0.067956,0.040773,...,0.027182,0.089497,0.130270,0.013591,0.000000,0.027182,0.021541,0.040773,0.013591,0.000000
AI597479,0.656463,0.643428,0.649865,0.811693,0.921158,0.708605,0.649573,0.515508,0.900329,0.835433,...,0.530076,0.512020,0.529186,0.815232,0.557236,0.896766,0.569827,0.638340,0.520060,0.452464
Bzw1,0.851849,0.859307,1.009986,1.107764,1.200532,0.966306,0.907786,0.757329,1.110300,1.129532,...,1.112185,0.840208,0.883291,0.760660,0.894612,1.131841,1.019013,0.875613,0.648975,0.941322
Fcamr,0.000000,0.000000,0.013591,0.000000,0.000000,0.000000,0.000000,0.000000,0.013591,0.000000,...,0.000000,0.021541,0.013591,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [474]:
matsmoothvals.to_csv('foxa1_data/smoothed_rna_table_G2_40.csv')

In [357]:
cell_ids.to_csv('foxa1_data/smoothed_dorc_columns_G2.csv', header = False)